In [1]:
import os
import pandas as pd
import glob
import nbformat

In [2]:
directory = 'C:/Users/FANNY MARIA D SOUZA/Capstone_Movement/Movement_CompletedSteps_SimpleStew/'
file_pattern = directory + '*_Movement.ipynb'

#### Extracting Position Features

In [3]:
def extract_features_summary(file_path):
    with open(file_path, 'r') as file:
        nb = nbformat.read(file, as_version=4)
    
    local_context = {}
    for cell in nb.cells:
        if cell.cell_type == 'code':
            try:
                # Check if the cell contains plotting code and skip it
                if 'plt.show()' in cell.source or 'plt.plot' in cell.source or 'plt.bar' in cell.source:
                    continue
                exec(cell.source, {}, local_context)
                if 'total_distances_df' in local_context:
                    return local_context['total_distances_df']
            except Exception as e:
                print(f"Error executing cell in {file_path}: {e}")
    return None

total_distance = []

# Extract data from each notebook
for file_path in glob.glob(file_pattern):
    participant_id = os.path.basename(file_path)[:4] 
    df = extract_features_summary(file_path)
    if df is not None:
        df['Participant'] = participant_id 
        total_distance.append(df)
    else:
        print(f"No features_summary found in {file_path}")

combined_features_summary = pd.concat(total_distance, ignore_index=True)
combined_features_summary

,Tracker,Total Distance,Participant
0,Chest,3412.670879,C001
1,LeftShoulder,3973.000007,C001
2,RightShoulder,0.000000,C001
3,LeftElbow,4288.582700,C001
4,RightElbow,0.000000,C001
...,...,...,...
123,LeftElbow,1450.211354,C032
124,RightElbow,6850.895792,C032
125,MotionControllerLeft,2304.416157,C032
126,MotionControllerRight,9826.000865,C032


In [4]:
pivoted_df = combined_features_summary.pivot_table(
    index='Participant', 
    columns='Tracker', 
    values='Total Distance'
).reset_index()

pivoted_df


Tracker,Participant,Chest,HeadCamera,LeftElbow,LeftShoulder,MotionControllerLeft,MotionControllerRight,RightElbow,RightShoulder
0,C001,3412.670879,4279.274901,4288.582700,3973.000007,5182.224529,10905.739544,0.000000,0.000000
1,C003,3924.459833,4188.586910,10956.856699,5295.321816,6069.771100,11752.435623,9980.064138,5986.207882
2,C006,1333.791569,3044.388844,1984.150383,2171.609415,3695.805905,10100.423327,6357.739703,4950.245383
3,C008,4242.098701,5457.755161,7109.453141,6251.381485,7812.578524,15305.334757,12542.308661,8928.642077
4,C009,6417.363081,7355.626722,9619.928363,8002.941907,10363.980246,17018.966026,13587.082828,9942.432144
5,C012,1885.463295,2343.370019,3102.490358,2282.249138,3587.356143,10589.813612,7435.435851,4836.915787
6,C013,6810.143228,7843.467810,10009.235997,9369.871433,12558.189528,15412.361778,12714.614154,10091.989281
7,C016,2344.234017,2511.131882,4298.854968,3616.865710,5000.861837,8592.558163,6352.376486,4692.583555
8,C017,3142.145291,4675.839669,4934.165755,4271.509320,6913.017052,11519.019822,9631.877375,6059.969241
9,C020,3334.669489,5200.170016,4989.299953,4342.714386,6011.462516,11887.168528,9021.289379,6577.576007


In [6]:
from sklearn.impute import KNNImputer
import numpy as np

imputer = KNNImputer(n_neighbors=1)

numerical_cols = pivoted_df.select_dtypes(include=['float64', 'int64']).columns
pivoted_df[numerical_cols] = pivoted_df[numerical_cols].replace(0, np.nan)

pivoted_df[numerical_cols] = imputer.fit_transform(pivoted_df[numerical_cols])
pivoted_df

Tracker,Participant,Chest,HeadCamera,LeftElbow,LeftShoulder,MotionControllerLeft,MotionControllerRight,RightElbow,RightShoulder
0,C001,3412.670879,4279.274901,4288.582700,3973.000007,5182.224529,10905.739544,9021.289379,6577.576007
1,C003,3924.459833,4188.586910,10956.856699,5295.321816,6069.771100,11752.435623,9980.064138,5986.207882
2,C006,1333.791569,3044.388844,1984.150383,2171.609415,3695.805905,10100.423327,6357.739703,4950.245383
3,C008,4242.098701,5457.755161,7109.453141,6251.381485,7812.578524,15305.334757,12542.308661,8928.642077
4,C009,6417.363081,7355.626722,9619.928363,8002.941907,10363.980246,17018.966026,13587.082828,9942.432144
5,C012,1885.463295,2343.370019,3102.490358,2282.249138,3587.356143,10589.813612,7435.435851,4836.915787
6,C013,6810.143228,7843.467810,10009.235997,9369.871433,12558.189528,15412.361778,12714.614154,10091.989281
7,C016,2344.234017,2511.131882,4298.854968,3616.865710,5000.861837,8592.558163,6352.376486,4692.583555
8,C017,3142.145291,4675.839669,4934.165755,4271.509320,6913.017052,11519.019822,9631.877375,6059.969241
9,C020,3334.669489,5200.170016,4989.299953,4342.714386,6011.462516,11887.168528,9021.289379,6577.576007


In [7]:
new_column_names = {
    'Chest': 'Chest Total Distance',
    'HeadCamera': 'HeadCamera Total Distance',
    'LeftElbow': 'LeftElbow Total Distance',
    'LeftShoulder': 'LeftShoulder Total Distance',
    'MotionControllerLeft': 'MotionControllerLeft Total Distance',
    'MotionControllerRight': 'MotionControllerRight Total Distance',
    'RightElbow': 'RightElbow Total Distance',
    'RightShoulder': 'RightShoulder Total Distance'
}

# Rename columns
pivoted_df = pivoted_df.rename(columns=new_column_names)
pivoted_df

Tracker,Participant,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance
0,C001,3412.670879,4279.274901,4288.582700,3973.000007,5182.224529,10905.739544,9021.289379,6577.576007
1,C003,3924.459833,4188.586910,10956.856699,5295.321816,6069.771100,11752.435623,9980.064138,5986.207882
2,C006,1333.791569,3044.388844,1984.150383,2171.609415,3695.805905,10100.423327,6357.739703,4950.245383
3,C008,4242.098701,5457.755161,7109.453141,6251.381485,7812.578524,15305.334757,12542.308661,8928.642077
4,C009,6417.363081,7355.626722,9619.928363,8002.941907,10363.980246,17018.966026,13587.082828,9942.432144
5,C012,1885.463295,2343.370019,3102.490358,2282.249138,3587.356143,10589.813612,7435.435851,4836.915787
6,C013,6810.143228,7843.467810,10009.235997,9369.871433,12558.189528,15412.361778,12714.614154,10091.989281
7,C016,2344.234017,2511.131882,4298.854968,3616.865710,5000.861837,8592.558163,6352.376486,4692.583555
8,C017,3142.145291,4675.839669,4934.165755,4271.509320,6913.017052,11519.019822,9631.877375,6059.969241
9,C020,3334.669489,5200.170016,4989.299953,4342.714386,6011.462516,11887.168528,9021.289379,6577.576007


In [8]:
pivoted_df.to_csv('PositionDistance_SimpleStew.csv',index=False)

In [9]:
bscore=pd.read_csv('M_BDScore.csv')

In [10]:
final_df = pd.merge(bscore, pivoted_df, on='Participant', how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance
0,C001,36,3412.670879,4279.274901,4288.582700,3973.000007,5182.224529,10905.739544,9021.289379,6577.576007
1,C003,58,3924.459833,4188.586910,10956.856699,5295.321816,6069.771100,11752.435623,9980.064138,5986.207882
2,C006,42,1333.791569,3044.388844,1984.150383,2171.609415,3695.805905,10100.423327,6357.739703,4950.245383
3,C008,34,4242.098701,5457.755161,7109.453141,6251.381485,7812.578524,15305.334757,12542.308661,8928.642077
4,C009,24,6417.363081,7355.626722,9619.928363,8002.941907,10363.980246,17018.966026,13587.082828,9942.432144
5,C012,54,1885.463295,2343.370019,3102.490358,2282.249138,3587.356143,10589.813612,7435.435851,4836.915787
6,C013,48,6810.143228,7843.467810,10009.235997,9369.871433,12558.189528,15412.361778,12714.614154,10091.989281
7,C016,39,2344.234017,2511.131882,4298.854968,3616.865710,5000.861837,8592.558163,6352.376486,4692.583555
8,C017,36,3142.145291,4675.839669,4934.165755,4271.509320,6913.017052,11519.019822,9631.877375,6059.969241
9,C020,30,3334.669489,5200.170016,4989.299953,4342.714386,6011.462516,11887.168528,9021.289379,6577.576007


In [11]:
addbeef=pd.read_csv('SimpleStew_AddBeef.csv')

In [12]:
if 'Block Design Raw Score' in addbeef.columns:
    addbeef['Block Design Raw Score'] = addbeef['Block Design Raw Score'].astype(int)
addbeef = addbeef.drop(columns=['Step'])

final_df = pd.merge(final_df, addbeef, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,Chest Distance - Add Beef,HeadCamera Distance - Add Beef,LeftElbow Distance - Add Beef,LeftShoulder Distance - Add Beef,MotionControllerLeft Distance - Add Beef,MotionControllerRight Distance - Add Beef,RightElbow Distance - Add Beef,RightShoulder Distance - Add Beef
0,C001,36,3412.670879,4279.274901,4288.582700,3973.000007,5182.224529,10905.739544,9021.289379,6577.576007,667.417289,892.612437,546.694850,632.755838,654.999073,2123.031272,1345.054914,908.199798
1,C003,58,3924.459833,4188.586910,10956.856699,5295.321816,6069.771100,11752.435623,9980.064138,5986.207882,244.553964,672.620084,3621.467395,1107.684830,1368.388019,1709.467574,1416.820567,896.141024
2,C006,42,1333.791569,3044.388844,1984.150383,2171.609415,3695.805905,10100.423327,6357.739703,4950.245383,258.659127,527.514825,377.353139,309.910282,500.435944,1480.011144,1066.727245,823.563443
3,C008,34,4242.098701,5457.755161,7109.453141,6251.381485,7812.578524,15305.334757,12542.308661,8928.642077,587.049964,823.307794,1251.138637,1007.294516,1487.661164,2516.994520,1975.579279,1316.243411
4,C009,24,6417.363081,7355.626722,9619.928363,8002.941907,10363.980246,17018.966026,13587.082828,9942.432144,1238.803190,1785.056217,3232.513404,2476.108557,3746.344524,4081.032920,2965.975129,2162.301897
5,C012,54,1885.463295,2343.370019,3102.490358,2282.249138,3587.356143,10589.813612,7435.435851,4836.915787,454.130793,584.809488,557.451996,426.532464,627.499578,2231.839961,1613.999599,1077.788100
6,C013,48,6810.143228,7843.467810,10009.235997,9369.871433,12558.189528,15412.361778,12714.614154,10091.989281,1167.707691,1268.056816,1461.985572,1396.548562,1764.663897,2825.347528,2306.200593,1748.974037
7,C016,39,2344.234017,2511.131882,4298.854968,3616.865710,5000.861837,8592.558163,6352.376486,4692.583555,807.781746,489.997092,962.516088,832.312275,1157.475116,1998.332420,1317.040787,964.063194
8,C017,36,3142.145291,4675.839669,4934.165755,4271.509320,6913.017052,11519.019822,9631.877375,6059.969241,389.851366,491.967770,586.114488,490.774556,899.433660,1360.655401,1129.551881,702.223280
9,C020,30,3334.669489,5200.170016,4989.299953,4342.714386,6011.462516,11887.168528,9021.289379,6577.576007,290.517401,426.004904,435.934120,386.303513,568.289623,1172.890210,832.428880,610.737268


In [13]:
addcarrot=pd.read_csv('SimpleStew_AddCarrot.csv')

In [14]:
if 'Block Design Raw Score' in addcarrot.columns:
    addcarrot['Block Design Raw Score'] = addcarrot['Block Design Raw Score'].astype(int)
addcarrot = addcarrot.drop(columns=['Step'])

final_df = pd.merge(final_df, addcarrot, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Beef,RightShoulder Distance - Add Beef,Chest Distance - Add Carrot,HeadCamera Distance - Add Carrot,LeftElbow Distance - Add Carrot,LeftShoulder Distance - Add Carrot,MotionControllerLeft Distance - Add Carrot,MotionControllerRight Distance - Add Carrot,RightElbow Distance - Add Carrot,RightShoulder Distance - Add Carrot
0,C001,36,3412.670879,4279.274901,4288.582700,3973.000007,5182.224529,10905.739544,9021.289379,6577.576007,...,1345.054914,908.199798,288.999699,379.583666,865.589347,584.494307,1058.248780,1419.609542,918.493993,677.122371
1,C003,58,3924.459833,4188.586910,10956.856699,5295.321816,6069.771100,11752.435623,9980.064138,5986.207882,...,1416.820567,896.141024,342.419485,393.899984,1100.342364,516.583023,994.127968,1303.629360,870.251108,627.740690
2,C006,42,1333.791569,3044.388844,1984.150383,2171.609415,3695.805905,10100.423327,6357.739703,4950.245383,...,1066.727245,823.563443,242.227829,497.650444,357.158209,437.660008,1105.580046,2196.767365,1636.134402,1063.917884
3,C008,34,4242.098701,5457.755161,7109.453141,6251.381485,7812.578524,15305.334757,12542.308661,8928.642077,...,1975.579279,1316.243411,818.867019,990.337315,1028.366151,950.469346,1062.438061,3460.554270,2789.408117,1910.393651
4,C009,24,6417.363081,7355.626722,9619.928363,8002.941907,10363.980246,17018.966026,13587.082828,9942.432144,...,2965.975129,2162.301897,649.742863,966.420435,841.597911,738.418392,783.598291,3130.228025,2427.025741,1648.663520
5,C012,54,1885.463295,2343.370019,3102.490358,2282.249138,3587.356143,10589.813612,7435.435851,4836.915787,...,1613.999599,1077.788100,154.774876,228.369369,357.055523,251.000307,533.429921,1497.255158,901.943739,545.204334
6,C013,48,6810.143228,7843.467810,10009.235997,9369.871433,12558.189528,15412.361778,12714.614154,10091.989281,...,2306.200593,1748.974037,1109.909673,1233.385764,1987.304044,1717.143781,2655.640645,2106.048150,1773.162285,1461.976485
7,C016,39,2344.234017,2511.131882,4298.854968,3616.865710,5000.861837,8592.558163,6352.376486,4692.583555,...,1317.040787,964.063194,190.101702,260.823421,730.316233,547.717473,882.843151,1197.011647,939.562708,690.801469
8,C017,36,3142.145291,4675.839669,4934.165755,4271.509320,6913.017052,11519.019822,9631.877375,6059.969241,...,1129.551881,702.223280,331.277532,490.245922,811.733563,510.211697,1143.469149,1371.036530,1075.218827,721.418378
9,C020,30,3334.669489,5200.170016,4989.299953,4342.714386,6011.462516,11887.168528,9021.289379,6577.576007,...,832.428880,610.737268,334.760673,451.081132,783.782422,568.637991,982.269754,1316.764144,918.493993,677.122371


In [15]:
addtomato=pd.read_csv('SimpleStew_AddTomato.csv')

In [16]:
if 'Block Design Raw Score' in addtomato.columns:
    addtomato['Block Design Raw Score'] = addtomato['Block Design Raw Score'].astype(int)
addtomato = addtomato.drop(columns=['Step'])

final_df = pd.merge(final_df, addtomato, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Carrot,RightShoulder Distance - Add Carrot,Chest Distance - Add Tomato,HeadCamera Distance - Add Tomato,LeftElbow Distance - Add Tomato,LeftShoulder Distance - Add Tomato,MotionControllerLeft Distance - Add Tomato,MotionControllerRight Distance - Add Tomato,RightElbow Distance - Add Tomato,RightShoulder Distance - Add Tomato
0,C001,36,3412.670879,4279.274901,4288.582700,3973.000007,5182.224529,10905.739544,9021.289379,6577.576007,...,918.493993,677.122371,327.683900,393.891177,421.349493,372.080950,534.877430,1203.602812,777.323767,535.493811
1,C003,58,3924.459833,4188.586910,10956.856699,5295.321816,6069.771100,11752.435623,9980.064138,5986.207882,...,870.251108,627.740690,250.908024,284.972133,284.543720,392.954786,394.183705,1164.539144,777.323767,535.493811
2,C006,42,1333.791569,3044.388844,1984.150383,2171.609415,3695.805905,10100.423327,6357.739703,4950.245383,...,1636.134402,1063.917884,137.554449,292.522962,297.478422,222.144245,357.139249,1144.092457,582.568224,502.170734
3,C008,34,4242.098701,5457.755161,7109.453141,6251.381485,7812.578524,15305.334757,12542.308661,8928.642077,...,2789.408117,1910.393651,534.672787,641.997731,985.654779,776.299903,1146.152361,1622.516570,1321.637857,935.817569
4,C009,24,6417.363081,7355.626722,9619.928363,8002.941907,10363.980246,17018.966026,13587.082828,9942.432144,...,2427.025741,1648.663520,451.364274,575.407165,664.991445,555.346191,697.523323,1703.799053,1329.495930,926.331470
5,C012,54,1885.463295,2343.370019,3102.490358,2282.249138,3587.356143,10589.813612,7435.435851,4836.915787,...,901.943739,545.204334,183.311747,242.273252,250.696526,196.161621,252.344016,1383.085429,887.716949,549.527433
6,C013,48,6810.143228,7843.467810,10009.235997,9369.871433,12558.189528,15412.361778,12714.614154,10091.989281,...,1773.162285,1461.976485,561.596475,644.710724,864.969294,808.711832,1153.162999,1091.067796,988.353981,805.611137
7,C016,39,2344.234017,2511.131882,4298.854968,3616.865710,5000.861837,8592.558163,6352.376486,4692.583555,...,939.562708,690.801469,141.033597,289.054550,487.846186,349.765318,651.565978,828.996996,580.051229,377.989399
8,C017,36,3142.145291,4675.839669,4934.165755,4271.509320,6913.017052,11519.019822,9631.877375,6059.969241,...,1075.218827,721.418378,343.463921,466.145582,586.153305,467.016411,659.137626,1320.842585,973.924385,657.861979
9,C020,30,3334.669489,5200.170016,4989.299953,4342.714386,6011.462516,11887.168528,9021.289379,6577.576007,...,918.493993,677.122371,405.865918,548.718495,366.705711,409.731268,434.856843,1515.597054,1131.609960,802.384096


In [18]:
pourbroth=pd.read_csv('SimpleStew_PourBroth.csv')

In [19]:
if 'Block Design Raw Score' in pourbroth.columns:
    pourbroth['Block Design Raw Score'] = pourbroth['Block Design Raw Score'].astype(int)
pourbroth = pourbroth.drop(columns=['Step'])

final_df = pd.merge(final_df, pourbroth, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Tomato,RightShoulder Distance - Add Tomato,Chest Distance - Pour Broth,HeadCamera Distance - Pour Broth,LeftElbow Distance - Pour Broth,LeftShoulder Distance - Pour Broth,MotionControllerLeft Distance - Pour Broth,MotionControllerRight Distance - Pour Broth,RightElbow Distance - Pour Broth,RightShoulder Distance - Pour Broth
0,C001,36,3412.670879,4279.274901,4288.582700,3973.000007,5182.224529,10905.739544,9021.289379,6577.576007,...,777.323767,535.493811,812.608216,899.941076,891.861802,928.598749,1035.573134,1784.495819,1972.819190,1142.676652
1,C003,58,3924.459833,4188.586910,10956.856699,5295.321816,6069.771100,11752.435623,9980.064138,5986.207882,...,777.323767,535.493811,2416.319672,1549.437498,4811.979810,2256.734721,2130.161392,2685.000315,3558.918685,1652.556965
2,C006,42,1333.791569,3044.388844,1984.150383,2171.609415,3695.805905,10100.423327,6357.739703,4950.245383,...,582.568224,502.170734,292.258129,442.117670,478.237162,381.813594,585.134376,754.859616,519.044154,447.152372
3,C008,34,4242.098701,5457.755161,7109.453141,6251.381485,7812.578524,15305.334757,12542.308661,8928.642077,...,1321.637857,935.817569,559.441615,527.436941,837.240495,771.943406,895.050565,973.826839,924.394622,817.754168
4,C009,24,6417.363081,7355.626722,9619.928363,8002.941907,10363.980246,17018.966026,13587.082828,9942.432144,...,1329.495930,926.331470,2137.194232,1588.586316,2359.839116,1937.072180,2705.981105,2923.157215,2442.208647,1935.139796
5,C012,54,1885.463295,2343.370019,3102.490358,2282.249138,3587.356143,10589.813612,7435.435851,4836.915787,...,887.716949,549.527433,315.072198,230.934174,354.365370,328.456425,361.834158,596.616266,502.160235,429.612033
6,C013,48,6810.143228,7843.467810,10009.235997,9369.871433,12558.189528,15412.361778,12714.614154,10091.989281,...,988.353981,805.611137,409.458086,448.240233,617.592776,654.561793,620.873495,750.523459,586.061400,515.786550
7,C016,39,2344.234017,2511.131882,4298.854968,3616.865710,5000.861837,8592.558163,6352.376486,4692.583555,...,580.051229,377.989399,338.203302,249.061733,368.896023,365.448502,319.961403,518.009513,426.375796,395.608540
8,C017,36,3142.145291,4675.839669,4934.165755,4271.509320,6913.017052,11519.019822,9631.877375,6059.969241,...,973.924385,657.861979,881.600939,1227.435473,1280.485380,1074.349904,1172.399652,1609.534276,1972.819190,1142.676652
9,C020,30,3334.669489,5200.170016,4989.299953,4342.714386,6011.462516,11887.168528,9021.289379,6577.576007,...,1131.609960,802.384096,602.727035,1439.972193,1137.772789,962.371731,1104.466442,1414.660442,1095.627786,909.828670


In [21]:
stirstew=pd.read_csv('SimpleStew_StirStew.csv')

In [22]:
if 'Block Design Raw Score' in stirstew.columns:
    stirstew['Block Design Raw Score'] = stirstew['Block Design Raw Score'].astype(int)
stirstew = stirstew.drop(columns=['Step'])

final_df = pd.merge(final_df, stirstew, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Pour Broth,RightShoulder Distance - Pour Broth,Chest Distance - Stir Stew,HeadCamera Distance - Stir Stew,LeftElbow Distance - Stir Stew,LeftShoulder Distance - Stir Stew,MotionControllerLeft Distance - Stir Stew,MotionControllerRight Distance - Stir Stew,RightElbow Distance - Stir Stew,RightShoulder Distance - Stir Stew
0,C001,36,3412.670879,4279.274901,4288.582700,3973.000007,5182.224529,10905.739544,9021.289379,6577.576007,...,1972.819190,1142.676652,90.138358,134.120134,120.829925,124.009876,132.623559,445.504319,366.899345,233.573050
1,C003,58,3924.459833,4188.586910,10956.856699,5295.321816,6069.771100,11752.435623,9980.064138,5986.207882,...,3558.918685,1652.556965,40.532867,107.558008,34.801405,28.183456,35.260268,502.933011,265.485212,184.388790
2,C006,42,1333.791569,3044.388844,1984.150383,2171.609415,3695.805905,10100.423327,6357.739703,4950.245383,...,519.044154,447.152372,42.021392,157.365390,186.650365,49.179454,118.324919,674.257603,453.228289,283.039911
3,C008,34,4242.098701,5457.755161,7109.453141,6251.381485,7812.578524,15305.334757,12542.308661,8928.642077,...,924.394622,817.754168,239.412143,466.561259,498.906194,510.501063,454.237010,1191.492808,997.801533,734.339878
4,C009,24,6417.363081,7355.626722,9619.928363,8002.941907,10363.980246,17018.966026,13587.082828,9942.432144,...,2442.208647,1935.139796,180.462248,252.347436,291.379632,246.464791,261.803254,769.368553,639.724118,437.651870
5,C012,54,1885.463295,2343.370019,3102.490358,2282.249138,3587.356143,10589.813612,7435.435851,4836.915787,...,502.160235,429.612033,102.162579,132.246780,186.650365,137.935384,201.916075,762.627879,552.592354,325.503183
6,C013,48,6810.143228,7843.467810,10009.235997,9369.871433,12558.189528,15412.361778,12714.614154,10091.989281,...,586.061400,515.786550,1119.004957,1240.710926,1247.138569,1258.734392,1589.584368,2816.012172,2422.945311,1833.909073
7,C016,39,2344.234017,2511.131882,4298.854968,3616.865710,5000.861837,8592.558163,6352.376486,4692.583555,...,426.375796,395.608540,149.041946,195.893943,224.334038,224.642758,216.270730,770.165733,545.896340,398.532594
8,C017,36,3142.145291,4675.839669,4934.165755,4271.509320,6913.017052,11519.019822,9631.877375,6059.969241,...,1972.819190,1142.676652,54.388560,147.637918,54.865752,77.496420,114.798720,419.418539,281.667443,182.842891
9,C020,30,3334.669489,5200.170016,4989.299953,4342.714386,6011.462516,11887.168528,9021.289379,6577.576007,...,1095.627786,909.828670,98.024993,145.704809,99.133999,115.300442,117.882908,487.370570,366.899345,233.573050


In [23]:
final_df.to_csv('Features_SimpleStew_Movement.csv',index=False)